# Deep BSDE Solver for European Call Option Pricing

This notebook demonstrates the use of deep learning to solve Backward Stochastic Differential Equations (BSDEs) for pricing European call options.

## Overview

We use a neural network to approximate the solution to the BSDE that represents the call option pricing problem:

- **Underlying dynamics**: Geometric Brownian Motion
- **Option**: European Call with strike K
- **Method**: Deep BSDE solver (Han, Jentzen, E, 2018)

The BSDE formulation:
- $dY_t = -f(t, X_t, Y_t, Z_t)dt + Z_t dW_t$
- $Y_T = g(X_T) = \max(X_T - K, 0)$

where $f(t, X_t, Y_t, Z_t) = -rY_t$ for option pricing.

## 1. Setup and Imports

In [ ]:
# Install required packages (uncomment if needed)
# !pip install torch numpy scipy matplotlib pandas munch tqdm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal as normal
from scipy.stats import norm
import munch
from tqdm import tqdm
import time
import pandas as pd

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 2. Define the Equation Classes

We define the base equation class and the specific call option equation.

In [ ]:
class Equation(object):
    """Base class for defining PDE related function."""

    def __init__(self, eqn_config):
        self.dim = eqn_config.dim
        self.total_time = eqn_config.total_time
        self.num_time_interval = eqn_config.num_time_interval
        self.delta_t = self.total_time / self.num_time_interval
        self.sqrt_delta_t = np.sqrt(self.delta_t)
        self.y_init = None

    def sample(self, num_sample):
        """Sample forward SDE."""
        raise NotImplementedError

    def f_torch(self, t, x, y, z):
        """Generator function in the PDE."""
        raise NotImplementedError

    def g_torch(self, t, x):
        """Terminal condition of the PDE."""
        raise NotImplementedError

In [ ]:
class CallOption(Equation):
    """European Call Option under Black-Scholes model."""
    
    def __init__(self, eqn_config):
        super(CallOption, self).__init__(eqn_config)
        self.strike = eqn_config.strike
        self.x_init = np.ones(self.dim) * eqn_config.x_init
        self.sigma = eqn_config.sigma
        self.r = eqn_config.r
        self.useExplicit = False

    def sample(self, num_sample):
        """Sample paths using Euler-Maruyama or exact solution."""
        dw_sample = normal.rvs(size=[num_sample, self.dim, self.num_time_interval]) * self.sqrt_delta_t
        
        if self.dim == 1:
            dw_sample = np.expand_dims(dw_sample, axis=0)
            dw_sample = np.swapaxes(dw_sample, 0, 1)

        x_sample = np.zeros([num_sample, self.dim, self.num_time_interval + 1])
        x_sample[:, :, 0] = np.ones([num_sample, self.dim]) * self.x_init

        if self.useExplicit:
            factor = np.exp((self.r - (self.sigma**2) / 2) * self.delta_t)
            for i in range(self.num_time_interval):
                x_sample[:, :, i + 1] = (factor * np.exp(self.sigma * dw_sample[:, :, i])) * x_sample[:, :, i]
        else:
            for i in range(self.num_time_interval):
                x_sample[:, :, i + 1] = ((1 + self.r * self.delta_t) * x_sample[:, :, i] + 
                                         (self.sigma * x_sample[:, :, i] * dw_sample[:, :, i]))

        return dw_sample, x_sample

    def f_torch(self, t, x, y, z):
        """Generator function: -rY for option pricing."""
        return -self.r * y

    def g_torch(self, t, x):
        """Terminal condition: max(S_T - K, 0)."""
        return torch.maximum(x - self.strike, torch.zeros_like(x))

    def SolExact(self, t, x):
        """Black-Scholes exact solution."""
        tau = self.total_time - t
        d1 = (np.log(x / self.strike) + (self.r + 0.5 * self.sigma**2) * tau) / (self.sigma * np.sqrt(tau))
        d2 = d1 - self.sigma * np.sqrt(tau)
        
        call = x * norm.cdf(d1) - self.strike * np.exp(-self.r * tau) * norm.cdf(d2)
        return call

## 3. Define the Neural Network Architecture

The neural network approximates the gradient $Z_t$ at each time step.

In [ ]:
DELTA_CLIP = 50.0

class FeedForwardSubNet(nn.Module):
    """Feedforward subnet for approximating Z at each time step."""
    
    def __init__(self, config, dim):
        super(FeedForwardSubNet, self).__init__()
        
        num_hiddens = config.net_config.num_hiddens
        
        if config.net_config.dtype == "float64":
            self.dtype = torch.float64
        else:
            self.dtype = torch.float32
        
        # Batch normalization layers
        self.bn_layers = nn.ModuleList([
            nn.BatchNorm1d(dim if i == 0 else num_hiddens[i-1] if i <= len(num_hiddens) else dim,
                          momentum=0.01, eps=1e-6)
            for i in range(len(num_hiddens) + 2)
        ])
        
        # Dense layers
        self.dense_layers = nn.ModuleList([
            nn.Linear(dim if i == 0 else num_hiddens[i-1], num_hiddens[i], bias=False)
            for i in range(len(num_hiddens))
        ])
        
        # Final output layer
        input_dim = num_hiddens[-1] if num_hiddens else dim
        self.dense_layers.append(nn.Linear(input_dim, dim, bias=True))

    def forward(self, x, training):
        """Forward pass: bn -> (dense -> bn -> relu) * n -> dense."""
        if training:
            self.train()
        else:
            self.eval()
        
        x = self.bn_layers[0](x)
        
        for i in range(len(self.dense_layers) - 1):
            x = self.dense_layers[i](x)
            x = self.bn_layers[i + 1](x)
            x = torch.relu(x)
        
        x = self.dense_layers[-1](x)
        return x

In [ ]:
class NonsharedModel(nn.Module):
    """Main BSDE solver model."""
    
    def __init__(self, config, bsde):
        super(NonsharedModel, self).__init__()
        self.config = config
        self.eqn_config = config.eqn_config
        self.net_config = config.net_config
        self.bsde = bsde
        self.dim = bsde.dim
        
        if self.net_config.dtype == "float64":
            self.dtype = torch.float64
        else:
            self.dtype = torch.float32
        
        # Learnable initial value Y_0
        self.y_init = nn.Parameter(
            torch.tensor(
                np.random.uniform(low=self.net_config.y_init_range[0],
                                high=self.net_config.y_init_range[1],
                                size=[1]),
                dtype=self.dtype
            )
        )
        
        # Learnable initial gradient Z_0
        self.z_init = nn.Parameter(
            torch.tensor(
                np.random.uniform(low=-.1, high=.1, size=[1, self.eqn_config.dim]),
                dtype=self.dtype
            )
        )
        
        # Subnet at each time step (except last)
        self.subnet = nn.ModuleList([
            FeedForwardSubNet(config, bsde.dim)
            for _ in range(self.bsde.num_time_interval - 1)
        ])

    def forward(self, inputs, training):
        """Forward simulation of the BSDE."""
        dw, x = inputs
        time_stamp = np.arange(0, self.eqn_config.num_time_interval) * self.bsde.delta_t
        
        batch_size = dw.shape[0]
        all_one_vec = torch.ones(batch_size, 1, dtype=self.dtype, device=dw.device)
        
        y = all_one_vec * self.y_init
        z = torch.matmul(all_one_vec, self.z_init)
        
        for t in range(0, self.bsde.num_time_interval - 1):
            f_val = self.bsde.f_torch(time_stamp[t], x[:, :, t], y, z)
            y = y - self.bsde.delta_t * f_val + torch.sum(z * dw[:, :, t], 1, keepdim=True)
            z = self.subnet[t](x[:, :, t + 1], training) / self.bsde.dim
        
        # Terminal time step
        f_val = self.bsde.f_torch(time_stamp[-1], x[:, :, -2], y, z)
        y = y - self.bsde.delta_t * f_val + torch.sum(z * dw[:, :, -1], 1, keepdim=True)
        
        return y

    def simulate_path(self, sample_data):
        """Simulate the entire path Y_0, Y_1, ..., Y_T."""
        self.eval()
        with torch.no_grad():
            dw, x = sample_data
            
            dw_t = torch.tensor(dw, dtype=self.dtype, device=self.y_init.device)
            x_t = torch.tensor(x, dtype=self.dtype, device=self.y_init.device)
            
            time_stamp = np.arange(0, self.eqn_config.num_time_interval) * self.bsde.delta_t
            batch_size = dw_t.shape[0]
            all_one_vec = torch.ones(batch_size, 1, dtype=self.dtype, device=dw_t.device)
            
            y = all_one_vec * self.y_init
            z = torch.matmul(all_one_vec, self.z_init)
            
            history = [y.unsqueeze(-1)]
            
            for t in range(0, self.bsde.num_time_interval - 1):
                f_val = self.bsde.f_torch(time_stamp[t], x_t[:, :, t], y, z)
                y = y - self.bsde.delta_t * f_val + torch.sum(z * dw_t[:, :, t], 1, keepdim=True)
                history.append(y.unsqueeze(-1))
                z = self.subnet[t](x_t[:, :, t + 1], False) / self.bsde.dim
            
            # Terminal time
            f_val = self.bsde.f_torch(time_stamp[-1], x_t[:, :, -2], y, z)
            y = y - self.bsde.delta_t * f_val + torch.sum(z * dw_t[:, :, -1], 1, keepdim=True)
            history.append(y.unsqueeze(-1))
            
            history = torch.cat(history, dim=-1)
        
        return history.cpu().numpy()

## 4. Define the Solver

In [ ]:
class BSDESolver(object):
    """The BSDE solver using deep neural networks."""
    
    def __init__(self, config, bsde):
        self.eqn_config = config.eqn_config
        self.net_config = config.net_config
        self.bsde = bsde
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        if self.net_config.dtype == "float64":
            self.dtype = torch.float64
            torch.set_default_dtype(torch.float64)
        else:
            self.dtype = torch.float32
            torch.set_default_dtype(torch.float32)
        
        self.model = NonsharedModel(config, bsde).to(self.device)
        
        self.lr_values = self.net_config.lr_values
        self.lr_boundaries = self.net_config.lr_boundaries
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.net_config.lr_values[0], eps=1e-8)

    def get_lr(self, step):
        """Piecewise constant learning rate schedule."""
        for i, boundary in enumerate(self.lr_boundaries):
            if step < boundary:
                return self.lr_values[i]
        return self.lr_values[-1]

    def loss_fn(self, inputs, training):
        """Compute the loss function."""
        dw, x = inputs
        
        dw_t = torch.tensor(dw, dtype=self.dtype, device=self.device)
        x_t = torch.tensor(x, dtype=self.dtype, device=self.device)
        
        y_terminal = self.model((dw_t, x_t), training)
        g_terminal = self.bsde.g_torch(self.bsde.total_time, x_t[:, :, -1])
        
        delta = y_terminal - g_terminal
        
        # Huber-like loss with clipping
        loss = torch.mean(torch.where(torch.abs(delta) < DELTA_CLIP,
                                      torch.square(delta),
                                      2 * DELTA_CLIP * torch.abs(delta) - DELTA_CLIP ** 2))
        
        # Penalty for Y_0 outside initial range
        loss += 1000 * (torch.maximum(self.model.y_init[0] - self.net_config.y_init_range[1],
                                     torch.tensor(0., dtype=self.dtype, device=self.device)) +
                       torch.maximum(self.net_config.y_init_range[0] - self.model.y_init[0],
                                    torch.tensor(0., dtype=self.dtype, device=self.device)))
        
        return loss

    def train(self):
        """Training loop."""
        start_time = time.time()
        training_history = []
        valid_data = self.bsde.sample(self.net_config.valid_size)

        for step in tqdm(range(self.net_config.num_iterations + 1)):
            # Update learning rate
            current_lr = self.get_lr(step)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = current_lr
            
            # Logging
            if step % self.net_config.logging_frequency == 0:
                loss = self.loss_fn(valid_data, training=False).item()
                y_init = self.model.y_init.detach().cpu().numpy()[0]
                elapsed_time = time.time() - start_time
                training_history.append([step, loss, y_init, elapsed_time])
                
                if self.net_config.verbose:
                    print(f"step: {step:5d}, loss: {loss:.4e}, Y0: {y_init:.4e}, elapsed time: {int(elapsed_time):3d}s")
            
            # Training step
            self.model.train()
            self.optimizer.zero_grad()
            train_data = self.bsde.sample(self.net_config.batch_size)
            loss = self.loss_fn(train_data, training=True)
            loss.backward()
            self.optimizer.step()
        
        return np.array(training_history)

## 5. Configure the Problem

Set up the call option pricing problem parameters.

In [ ]:
# Problem parameters
dim = 1                    # Dimension (1D for single asset)
P = 2048                   # Number of Monte Carlo paths for evaluation
batch_size = 64            # Training batch size
total_time = 1.0           # Maturity (1 year)
num_time_interval = 200    # Number of time steps
strike = 100               # Strike price
r = 0.01                   # Risk-free rate (1%)
sigma = 0.25               # Volatility (25%)
x_init = 100               # Initial stock price

config = {
    "eqn_config": {
        "_comment": "European call option",
        "eqn_name": "CallOption",
        "total_time": total_time,
        "dim": dim,
        "num_time_interval": num_time_interval,
        "strike": strike,
        "r": r,
        "sigma": sigma,
        "x_init": x_init,
    },
    "net_config": {
        "y_init_range": [9, 11],                      # Initial guess range for option price
        "num_hiddens": [dim + 20, dim + 20],          # Hidden layer sizes
        "lr_values": [5e-2, 5e-3],                    # Learning rates
        "lr_boundaries": [2000],                      # LR schedule boundaries
        "num_iterations": 4000,                       # Training iterations
        "batch_size": batch_size,
        "valid_size": 1024,                           # Validation set size
        "logging_frequency": 100,                     # Log every N steps
        "dtype": "float64",                           # Use double precision
        "verbose": True
    }
}

config = munch.munchify(config)

print("Configuration:")
print(f"  Stock price S0 = ${x_init}")
print(f"  Strike K = ${strike}")
print(f"  Volatility σ = {sigma*100}%")
print(f"  Risk-free rate r = {r*100}%")
print(f"  Maturity T = {total_time} year")
print(f"  Time steps = {num_time_interval}")

## 6. Compute Black-Scholes Exact Solution

For comparison, we compute the exact Black-Scholes price.

In [ ]:
# Initialize the equation
bsde = CallOption(config.eqn_config)

# Compute exact Black-Scholes price
exact_price = bsde.SolExact(0, x_init)

print(f"\nBlack-Scholes Exact Price: ${exact_price:.4f}")

## 7. Train the Deep BSDE Solver

In [ ]:
# Initialize solver
bsde_solver = BSDESolver(config, bsde)

print(f"\nModel has {sum(p.numel() for p in bsde_solver.model.parameters())} parameters")
print(f"Training on {bsde_solver.device}\n")

# Train the model
training_history = bsde_solver.train()

## 8. Analyze Training Results

In [ ]:
# Extract training history
steps = training_history[:, 0]
losses = training_history[:, 1]
y0_values = training_history[:, 2]
times = training_history[:, 3]

# Final results
final_y0 = y0_values[-1]
final_loss = losses[-1]
error = abs(final_y0 - exact_price)
relative_error = error / exact_price * 100

print(f"\n{'='*60}")
print("FINAL RESULTS")
print(f"{'='*60}")
print(f"Black-Scholes Price:    ${exact_price:.6f}")
print(f"Deep Solver Price (Y0): ${final_y0:.6f}")
print(f"Absolute Error:         ${error:.6f}")
print(f"Relative Error:         {relative_error:.4f}%")
print(f"Final Loss:             {final_loss:.6e}")
print(f"Total Training Time:    {times[-1]:.1f}s")
print(f"{'='*60}\n")

## 9. Plot Training Convergence

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Loss curve
axes[0].semilogy(steps, losses, 'b-', linewidth=2, label='Training Loss')
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training Loss Convergence', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)

# Plot 2: Y0 convergence
axes[1].plot(steps, y0_values, 'g-', linewidth=2, label='Deep Solver Y0')
axes[1].axhline(y=exact_price, color='r', linestyle='--', linewidth=2, label='Black-Scholes Exact')
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('Option Price ($)', fontsize=12)
axes[1].set_title('Y0 (Option Price) Convergence', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

## 10. Simulate Exposure Profiles

Simulate the option value along paths to compute Expected Positive Exposure (EPE) and Expected Negative Exposure (ENE).

In [ ]:
# Simulate paths
print("Simulating exposure paths...")
simulations = bsde_solver.model.simulate_path(bsde.sample(P))

# Calculate discounted exposures
time_stamp = np.linspace(0, 1, num_time_interval + 1)
discount_factors = np.exp(-r * time_stamp)

epe = np.mean(discount_factors * np.maximum(simulations, 0), axis=0)
ene = np.mean(discount_factors * np.minimum(simulations, 0), axis=0)

# Exact solution at each time point
epe_exact = np.array([exact_price] + [exact_price for s in time_stamp[1:]])
ene_exact = np.zeros_like(time_stamp)

print(f"Simulated {P} paths with {num_time_interval} time steps")

## 11. Plot Exposure Profiles

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

# Plot EPE
ax.plot(time_stamp, epe_exact, 'b--', linewidth=2.5, label='DEPE (Exact)', alpha=0.8)
ax.plot(time_stamp, epe[0], 'b-', linewidth=2, label='DEPE (Deep Solver)', alpha=0.9)

# Plot ENE
ax.plot(time_stamp, ene_exact, 'r--', linewidth=2.5, label='DNPE (Exact)', alpha=0.8)
ax.plot(time_stamp, ene[0], 'r-', linewidth=2, label='DNPE (Deep Solver)', alpha=0.9)

ax.set_xlabel('Time (years)', fontsize=13)
ax.set_ylabel('Discounted Exposure ($)', fontsize=13)
ax.set_title('Expected Positive and Negative Exposure Profiles\nCall Option (K=$100, S0=$100, σ=25%, r=1%)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Statistical Analysis of Simulations

In [ ]:
# Statistics at maturity
terminal_values = simulations[:, 0, -1]

print("\nStatistics at Maturity (T=1):")
print(f"  Mean:     ${np.mean(terminal_values):.4f}")
print(f"  Std Dev:  ${np.std(terminal_values):.4f}")
print(f"  Min:      ${np.min(terminal_values):.4f}")
print(f"  Max:      ${np.max(terminal_values):.4f}")
print(f"  Median:   ${np.median(terminal_values):.4f}")

# Percentiles
percentiles = [5, 25, 50, 75, 95]
print("\nPercentiles:")
for p in percentiles:
    print(f"  {p}th:     ${np.percentile(terminal_values, p):.4f}")

## 13. Plot Distribution of Terminal Values

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Histogram
n, bins, patches = ax.hist(terminal_values, bins=50, density=True, alpha=0.7, 
                           color='steelblue', edgecolor='black', linewidth=0.5)

# Add exact price line
ax.axvline(exact_price, color='red', linestyle='--', linewidth=2, 
           label=f'Black-Scholes: ${exact_price:.2f}')
ax.axvline(np.mean(terminal_values), color='green', linestyle='-', linewidth=2,
           label=f'Deep Solver Mean: ${np.mean(terminal_values):.2f}')

ax.set_xlabel('Option Value at Maturity ($)', fontsize=12)
ax.set_ylabel('Probability Density', fontsize=12)
ax.set_title('Distribution of Terminal Option Values', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 14. Export Results

In [ ]:
# Create DataFrame with training history
df_training = pd.DataFrame(training_history, columns=['Step', 'Loss', 'Y0', 'Time (s)'])

# Create DataFrame with exposure profiles
df_exposure = pd.DataFrame({
    'Time': time_stamp,
    'EPE_Exact': epe_exact,
    'EPE_DeepSolver': epe[0],
    'ENE_Exact': ene_exact,
    'ENE_DeepSolver': ene[0]
})

# Create DataFrame with simulation paths (first 10 paths)
df_simulations = pd.DataFrame(simulations[:10, 0, :].T, 
                              columns=[f'Path_{i+1}' for i in range(10)])
df_simulations.insert(0, 'Time', time_stamp)

print("\nDataFrames created:")
print(f"  Training history: {df_training.shape}")
print(f"  Exposure profiles: {df_exposure.shape}")
print(f"  Sample paths: {df_simulations.shape}")

# Display sample of training history
print("\nTraining History (last 5 rows):")
display(df_training.tail())

# Display exposure profile
print("\nExposure Profile (first 10 rows):")
display(df_exposure.head(10))

## 15. Save Results to Excel (Optional)

In [ ]:
# Uncomment to save results
# with pd.ExcelWriter('call_option_results.xlsx') as writer:
#     df_training.to_excel(writer, sheet_name='Training History', index=False)
#     df_exposure.to_excel(writer, sheet_name='Exposure Profiles', index=False)
#     df_simulations.to_excel(writer, sheet_name='Sample Paths', index=False)
# 
# print("Results saved to 'call_option_results.xlsx'")

## 16. Model Information and Summary

In [ ]:
print("\n" + "="*70)
print("MODEL SUMMARY")
print("="*70)
print(f"\nProblem Configuration:")
print(f"  Type:              European Call Option")
print(f"  Dimension:         {dim}")
print(f"  Time intervals:    {num_time_interval}")
print(f"  Total time:        {total_time} year")
print(f"\nMarket Parameters:")
print(f"  Initial price:     ${x_init}")
print(f"  Strike:            ${strike}")
print(f"  Volatility:        {sigma*100}%")
print(f"  Risk-free rate:    {r*100}%")
print(f"\nNetwork Architecture:")
print(f"  Hidden layers:     {config.net_config.num_hiddens}")
print(f"  Parameters:        {sum(p.numel() for p in bsde_solver.model.parameters()):,}")
print(f"  Dtype:             {config.net_config.dtype}")
print(f"  Device:            {bsde_solver.device}")
print(f"\nTraining Configuration:")
print(f"  Iterations:        {config.net_config.num_iterations}")
print(f"  Batch size:        {config.net_config.batch_size}")
print(f"  Learning rates:    {config.net_config.lr_values}")
print(f"  LR boundaries:     {config.net_config.lr_boundaries}")
print(f"\nResults:")
print(f"  Black-Scholes:     ${exact_price:.6f}")
print(f"  Deep Solver:       ${final_y0:.6f}")
print(f"  Absolute Error:    ${error:.6f}")
print(f"  Relative Error:    {relative_error:.4f}%")
print(f"  Training time:     {times[-1]:.1f}s")
print("="*70 + "\n")

## Conclusion

This notebook demonstrated:

1. **Deep BSDE Solver**: Neural network approximation of the solution to backward stochastic differential equations
2. **Call Option Pricing**: Application to European call option under Black-Scholes model
3. **Validation**: Comparison with exact Black-Scholes formula
4. **Exposure Analysis**: Computation of expected positive and negative exposure profiles

The deep learning approach successfully approximates the option price with high accuracy, validating the method for more complex problems where analytical solutions are not available.

### References

- Han, J., Jentzen, A., & E, W. (2018). Solving high-dimensional partial differential equations using deep learning. PNAS.
- Gnoatto, A., Picarelli, A., & Reisinger, C. (2020). Deep xVA solver. arXiv:2005.02633.